In [77]:
import os
import requests
import pandas as pd
import yfinance as yf
from time import sleep
from selenium import webdriver

In [78]:
# Chrome Driver settings

curdir = os.getcwd()

chrome_options = webdriver.ChromeOptions()

prefs = {'download.default_directory' : curdir}
chrome_options.add_experimental_option('prefs', prefs)

print(curdir)

/home/ianaraujo/Projects/idiv-dividend-analysis


In [79]:
# Download IDIV index composition from B3

B3_URL = 'https://sistemaswebb3-listados.b3.com.br/indexPage/day/IDIV'

driver = webdriver.Chrome('./chromedriver', options=chrome_options)

def download_idiv(url):

    driver.get(url)

    download_button = driver.find_element_by_link_text('Download')

    download_button.click()

    sleep(1)

download_idiv(B3_URL)    
    
driver.close()

In [80]:
# Get list of stocks from IDIV index

idiv_file = [curdir + '/' + str(file) for file in os.listdir(curdir) if str.startswith(file, 'IDIV')]

df = pd.read_csv(idiv_file[0], sep=';', header=1, skipfooter=2, encoding='latin-1', engine='python')

lista_codigos_negociacao = list(df.index.values)

print(lista_codigos_negociacao)

['ABCB4', 'AESB3', 'ALUP11', 'B3SA3', 'BRSR6', 'BBSE3', 'BBDC3', 'BBDC4', 'BRAP4', 'BBAS3', 'AGRO3', 'CMIG3', 'CMIG4', 'CSMG3', 'CPLE3', 'CPLE6', 'CPFE3', 'CYRE3', 'DIRR3', 'ELET3', 'ELET6', 'ENAT3', 'ENBR3', 'EGIE3', 'FESA4', 'GRND3', 'MYPK3', 'ITSA4', 'JHSF3', 'LEVE3', 'BEEF3', 'PETR3', 'PSSA3', 'QUAL3', 'RAPT4', 'ROMI3', 'SANB11', 'CSNA3', 'SYNE3', 'TAEE11', 'TGMA3', 'VIVT3', 'TRPL4', 'TRIS3', 'UNIP6', 'VALE3', 'WIZS3']


In [81]:
# Delete downloaded .csv file

os.remove(idiv_file[0])

In [82]:
# 

STATUS_INVEST_URL = 'https://statusinvest.com.br/acao/companytickerprovents?ticker={}&chartProventsType=2'

def stock_earnings(stock):

    df = pd.DataFrame()

    for codigo in stock:

        resp = requests.get(STATUS_INVEST_URL.format(codigo))
        
        json = resp.json()

        data = pd.DataFrame(dict(json)['assetEarningsYearlyModels'])

        data['stock'] = codigo

        df = pd.concat([df, data])

    return df

dividendos = stock_earnings(lista_codigos_negociacao).reset_index()

dividendos.tail(30)

,index,rank,value,stock
596,8,2017,5.042105,UNIP6
597,9,2018,2.709793,UNIP6
598,10,2019,0.302965,UNIP6
599,11,2020,1.220858,UNIP6
600,12,2021,15.444590,UNIP6
601,13,2022,8.965346,UNIP6
602,0,2007,0.394109,VALE3
603,1,2008,1.089846,VALE3
604,2,2009,1.016622,VALE3
605,3,2010,0.984241,VALE3


In [85]:
#

dividendos = dividendos[['stock', 'rank', 'value']]

df_dividendos = dividendos.rename(columns={'rank': 'year', 'value': 'dividend_per_share'})

df_dividendos = df_dividendos.loc[df_dividendos['year'].isin([2021, 2020, 2019]), :]

df_dividendos_5y = df_dividendos.groupby('stock').agg({'dividend_per_share': 'mean'})

df_dividendos_5y['dividend_per_share'] = df_dividendos_5y.dividend_per_share.apply(lambda x: round(x, 2))

df_dividendos_5y.head(10)

,dividend_per_share
stock,
ABCB4,0.82
AESB3,0.23
AGRO3,1.42
ALUP11,0.66
B3SA3,0.69
BBAS3,2.10
BBDC3,1.14
BBDC4,1.25
BBSE3,1.79


In [90]:
# Obter as cotações atualizadas para calcular o Dividend Yield

codigos_sa = [codigo + '.SA' for codigo in lista_codigos_negociacao]

cotacoes = map(lambda x: yf.Ticker(x).info['regularMarketPrice'], codigos_sa)

lista_cotacoes = list(cotacoes)

df_market_price = pd.DataFrame({'stock': lista_codigos_negociacao, 'market_price': lista_cotacoes})

df_market_price.head(10)

,stock,market_price
0,ABCB4,18.96
1,AESB3,9.82
2,ALUP11,28.82
3,B3SA3,12.43
4,BRSR6,11.07
5,BBSE3,28.50
6,BBDC3,16.12
7,BBDC4,19.48
8,BRAP4,23.65
9,BBAS3,41.47


In [91]:
df_dividend2 = df_dividendos_5y.merge(df_market_price, on='stock')

df_dividend2['dividend_yield'] = round((df_dividend2['dividend_per_share']/df_dividend2['market_price'])*100, 2)

df_dividend2

,stock,dividend_per_share,market_price,dividend_yield
0,ABCB4,0.82,18.96,4.32
1,AESB3,0.23,9.82,2.34
2,AGRO3,1.42,27.59,5.15
3,ALUP11,0.66,28.82,2.29
4,B3SA3,0.69,12.43,5.55
5,BBAS3,2.10,41.47,5.06
6,BBDC3,1.14,16.12,7.07
7,BBDC4,1.25,19.48,6.42
8,BBSE3,1.79,28.50,6.28
9,BEEF3,0.69,14.69,4.70
